<a href="https://colab.research.google.com/github/aatika-hakim/Music-Compositor-Agent/blob/main/Music_Compositor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain langchain_groq langchain_openai langgraph music21 python-dotenv gtts PyPDF2 pygame

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.0/118.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 24.7 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.13
    Uninstalling langchain-core-0.3.13:
      Successfully uninstalled langchain-core-0.3.13


In [ ]:
from typing import Dict, TypedDict
from langgraph.graph import StateGraph, END
from langchain.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from google.colab import userdata
import music21
import tempfile
import os
import random
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
api_key: str = userdata.get('GROQ_API_KEY')

# Initialize the Language Model (LLM) for generating musical components
llm = ChatGroq(model="llama-3.1-70b-versatile", temperature=0, api_key=api_key)

# State Definition
class MusicState(TypedDict):
    """Define the structure of the state for the music generation workflow."""
    musician_input: str  # User's input describing the desired music
    melody: str          # Generated melody
    harmony: str         # Generated harmony
    rhythm: str          # Generated rhythm
    style: str           # Desired musical style
    composition: str     # Complete musical composition
    midi_file: str       # Path to the generated MIDI file

# Component Functions
def melody_generator(state: MusicState) -> Dict:
    """Generate a melody based on the user's input."""
    prompt = ChatPromptTemplate.from_template(
        "Generate a melody based on this input: {input}. Represent it as a string of notes in music21 format."
    )
    chain = prompt | llm
    melody = chain.invoke({"input": state["musician_input"]})
    return {"melody": melody.content}

def harmony_creator(state: MusicState) -> Dict:
    """Create harmony for the generated melody."""
    prompt = ChatPromptTemplate.from_template(
        "Create harmony for this melody: {melody}. Represent it as a string of chords in music21 format."
    )
    chain = prompt | llm
    harmony = chain.invoke({"melody": state["melody"]})
    return {"harmony": harmony.content}

def rhythm_analyzer(state: MusicState) -> Dict:
    """Analyze and suggest a rhythm for the melody and harmony."""
    prompt = ChatPromptTemplate.from_template(
        "Analyze and suggest a rhythm for this melody and harmony: {melody}, {harmony}. Represent it as a string of durations in music21 format."
    )
    chain = prompt | llm
    rhythm = chain.invoke({"melody": state["melody"], "harmony": state["harmony"]})
    return {"rhythm": rhythm.content}

def style_adapter(state: MusicState) -> Dict:
    """Adapt the composition to the specified musical style."""
    prompt = ChatPromptTemplate.from_template(
        "Adapt this composition to the {style} style: Melody: {melody}, Harmony: {harmony}, Rhythm: {rhythm}. Provide the result in music21 format."
    )
    chain = prompt | llm
    adapted = chain.invoke({
        "style": state["style"],
        "melody": state["melody"],
        "harmony": state["harmony"],
        "rhythm": state["rhythm"]
    })
    return {"composition": adapted.content}

def midi_converter(state: MusicState) -> Dict:
    """Convert the composition to MIDI format and save it as a file."""
    piece = music21.stream.Score()

    # Create the melody and harmony from the state
    user_input = state['musician_input'].lower()
    scale_name = 'C major' if 'major' in user_input else 'C minor'

    melody = create_melody(scale_name, 7)
    harmony = create_chord_progression(7)

    piece.append(melody)
    piece.append(harmony)
    piece.insert(0, music21.tempo.MetronomeMark(number=60))

    # Use NamedTemporaryFile to create a temp MIDI file
    with tempfile.NamedTemporaryFile(delete=False, suffix='.mid') as temp_midi:
        piece.write('midi', temp_midi.name)
        midi_file_path = temp_midi.name  # Get the path of the temp file

    return {"midi_file": midi_file_path}


# Graph Construction
workflow = StateGraph(MusicState)

# Add nodes to the graph
workflow.add_node("melody_generator", melody_generator)
workflow.add_node("harmony_creator", harmony_creator)
workflow.add_node("rhythm_analyzer", rhythm_analyzer)
workflow.add_node("style_adapter", style_adapter)
workflow.add_node("midi_converter", midi_converter)

# Set the entry point of the graph
workflow.set_entry_point("melody_generator")

# Add edges to connect the nodes
workflow.add_edge("melody_generator", "harmony_creator")
workflow.add_edge("harmony_creator", "rhythm_analyzer")
workflow.add_edge("rhythm_analyzer", "style_adapter")
workflow.add_edge("style_adapter", "midi_converter")
workflow.add_edge("midi_converter", END)

# Compile the graph
app = workflow.compile()

# Invocation Example
inputs = {
    "musician_input": "Create a relaxing melody with a classical style",
    "style": "Classical"
}

result = app.invoke(inputs)
print("MIDI file created at:", result["midi_file"])




In [ ]:

import shutil

# Move the file to the current working directory with a new name
shutil.move('/tmp/tmpi1l5f6dw.mid', './music.mid')

In [ ]:
import os
from music21 import note, stream, tempo, chord
import tempfile
import random

scales = {
    'C major': ['C', 'D', 'E', 'F', 'G', 'A', 'B'],
    'C minor': ['C', 'D', 'Eb', 'F', 'G', 'Ab', 'Bb'],
}

chords = {
    'C major': ['C4', 'E4', 'G4'],
    'C minor': ['C4', 'Eb4', 'G4'],
}

def create_melody(scale_name, duration):
    melody = stream.Part()
    scale = scales[scale_name]
    for _ in range(duration):
        note_obj = note.Note(random.choice(scale) + '4')
        note_obj.quarterLength = 1
        melody.append(note_obj)
    return melody

def create_chord_progression(duration):
    """Create a simple chord progression."""
    harmony = stream.Part()
    for _ in range(duration):
        chord_name = random.choice(list(chords.keys()))
        chord_obj = chord.Chord(chords[chord_name])  # Use chord.Chord instead of stream.Chord
        chord_obj.quarterLength = 1
        harmony.append(chord_obj)
    return harmony

def generate_midi_file(scale_name="C major", duration=7):
    piece = stream.Score()
    melody = create_melody(scale_name, duration)
    harmony = create_chord_progression(duration)
    piece.append(melody)
    piece.append(harmony)
    piece.insert(0, tempo.MetronomeMark(number=60))

    # Use a temporary file path
    with tempfile.NamedTemporaryFile(delete=False, suffix='.mid') as temp_midi:
        file_path = temp_midi.name

    try:
        piece.write('midi', file_path)
        print("MIDI file created at:", file_path)
    except Exception as e:
        print("Error while creating MIDI file:", e)

# Run the function
generate_midi_file(scale_name="C major", duration=7)


In [ ]:
import shutil

# Move the file to the current working directory with a new name
shutil.move('/tmp/tmpqosx_5o_.mid', './your_music.mid')


In [ ]:
# Additional Imports
from typing import List

# Update MusicState to include lyrics and sections
class MusicState(TypedDict):
    musician_input: str
    style: str
    melody: str
    harmony: str
    rhythm: str
    lyrics: str
    composition: str
    midi_file: str

def lyrics_generator(state: MusicState) -> Dict:
    print("Generating lyrics...")
    prompt = ChatPromptTemplate.from_template(
        "Generate song lyrics based on this theme: {input}. Include verses and a chorus."
    )
    chain = prompt | llm
    lyrics = chain.invoke({"input": state["musician_input"]})
    print("Lyrics generated:", lyrics.content)
    return {"lyrics": lyrics.content}  # Updates 'lyrics'


def verse_creator(state: MusicState) -> Dict:
    print("Creating verse melody and harmony...")
    prompt = ChatPromptTemplate.from_template(
        "Generate a verse melody and harmony in {style} style for lyrics: {lyrics}. Use music21 format."
    )
    chain = prompt | llm
    verse = chain.invoke({"style": state["style"], "lyrics": state["lyrics"]})
    print("Verse created:", verse.content)
    # Ensure that 'melody' key is updated in MusicState
    return {"melody": verse.content}  # Updates 'melody'


def chorus_creator(state: MusicState) -> Dict:
    print("Creating chorus melody and harmony...")
    prompt = ChatPromptTemplate.from_template(
        "Generate a chorus melody and harmony in {style} style for lyrics: {lyrics}. Use music21 format."
    )
    chain = prompt | llm
    chorus = chain.invoke({"style": state["style"], "lyrics": state["lyrics"]})
    print("Chorus created:", chorus.content)
    # Ensure that 'harmony' key is updated in MusicState
    return {"harmony": chorus.content}  # Updates 'harmony'


def arrange_song(state: MusicState) -> Dict:
    print("Arranging song...")
    composition = f"Verse:\n{state['melody']}\nChorus:\n{state['harmony']}\n"
    print("Composition arranged:", composition)
    # Ensure that 'composition' key is updated in MusicState
    return {"composition": composition}  # Updates 'composition'


def midi_converter(state: MusicState) -> Dict:
    print("Converting to MIDI...")
    piece = music21.stream.Score()
    # Add sections
    verse_melody = create_melody('C major', 8)
    chorus_melody = create_melody('C major', 8)
    piece.append(verse_melody)
    piece.append(chorus_melody)
    piece.insert(0, music21.tempo.MetronomeMark(number=90))

    with tempfile.NamedTemporaryFile(delete=False, suffix='.mid') as temp_midi:
        piece.write('midi', temp_midi.name)
        midi_file_path = temp_midi.name
    print("MIDI file created:", midi_file_path)

    # Ensure that 'midi_file' key is updated in MusicState
    return {"midi_file": midi_file_path}  # Updates 'midi_file'



# Revised Graph Construction
# Reinitialize the StateGraph and Compile it after the function changes
workflow = StateGraph(MusicState)

# Add nodes for the full song generation
workflow.add_node("lyrics_generator", lyrics_generator)
workflow.add_node("verse_creator", verse_creator)
workflow.add_node("chorus_creator", chorus_creator)
workflow.add_node("arrange_song", arrange_song)
workflow.add_node("midi_converter", midi_converter)

# Set the entry point and define edges
workflow.set_entry_point("lyrics_generator")
workflow.add_edge("lyrics_generator", "verse_creator")
workflow.add_edge("verse_creator", "chorus_creator")
workflow.add_edge("chorus_creator", "arrange_song")
workflow.add_edge("arrange_song", "midi_converter")
workflow.add_edge("midi_converter", END) # Connect the last node to the END

# Compile the graph
app = workflow.compile()  # This was missing in the previous response

inputs = {
    "musician_input": "A peaceful, soulful love song in a classical style",
    "style": "Classical"
}

result = app.invoke(inputs)
print("Song MIDI file created at:", result["midi_file"])


In [ ]:
import shutil

# Move the file to the current working directory with a new name
shutil.move('/tmp/tmparrovzjy.mid', './song.mid')


In [ ]:
import shutil

# Move the file to the current working directory with a new name
shutil.move('/tmp/tmparrovzjy.mid', './test.mid')
